# Start

In [56]:
import importlib
import os
# import pandas as pd
# import json
# from datetime import datetime
from pathlib import Path
# from io import StringIO

from langchain_core.tools import Tool, StructuredTool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

from bsm_multi_agents.graph.state import WorkflowState

In [57]:
# from bsm_multi_agents.agents import utils
# importlib.reload(utils)
# from bsm_multi_agents.agents.utils import get_tool_result_from_messages,print_resp

In [58]:
cwd = Path.cwd()
project_path = cwd.parent.parent
project_path

PosixPath('/Users/yifanli/Github/model_doc_automation')

# Graph

In [60]:
from langgraph.graph import StateGraph, END, START

from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.pricing_calculator_nodes import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

graph = StateGraph(WorkflowState)
graph.add_node("pricing_calculator_agent", pricing_calculator_agent_node)
graph.add_node("pricing_calculator_tool", pricing_calculator_tool_node)

graph.add_edge(START, "pricing_calculator_agent")
graph.add_edge("pricing_calculator_agent", "pricing_calculator_tool")
graph.add_edge("pricing_calculator_tool", END)

app = graph.compile()

In [61]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
init_state = WorkflowState(
    csv_file_path=csv_file_path,
    output_dir=output_dir,
    server_path=server_path,
    # "remaining_steps": 10,
    # "messages": [HumanMessage(content=f"Load CSV from: {file_path}")],
)

final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

In [8]:
final_state

{'messages': [SystemMessage(content='\n    You are a quantitative calculator agent.\n    You have access to tools that call an external MCP server.\n    Each tool reads input file paths from the shared workflow state\n    and writes result file paths back into the state.\n    Do NOT ask for raw CSV content. Always work with paths only.\n    You do not need to provide any arguments to the tools. Just call them with {}.\n    ', additional_kwargs={}, response_metadata={}, id='95bba817-974b-4e7f-b8f5-46944359872e'),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` tool to calculate prices.\n- Use the `calculate_greeks_to_file` tool to calculate greeks.', additional_kwargs={}, response_metadata={}, id='a6738cfc-7874-409d-8c25-4f50dab1db9d'),
  AIMessage(content='{\n  "tool": "calculate_bsm_to_file",\n  "input_file": "/Users/yif

# Calculator

## MCP Tool Level

### calculate_bsm_to_file

In [9]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [12]:
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_bsm_to_file

In [13]:
calculate_bsm_to_file(csv_file_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'

### calculate_greeks_to_file

In [14]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [15]:
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_greeks_to_file

In [16]:
calculate_greeks_to_file(csv_file_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv'

## MCP Server Level

## Call MCP Level

### call_mcp_tool_async

In [17]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [18]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import call_mcp_tool_async

In [19]:
res = await call_mcp_tool_async(
    tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    arguments={
        "input_path": csv_file_path,
        "output_dir": output_dir,
    },
)
res

CallToolResult(meta=None, content=[TextContent(type='text', text='/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv', annotations=None, meta=None)], structuredContent={'result': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'}, isError=False)

### call_mcp_tool

### run_in_new_loop

### create_mcp_state_tool_wrapper

In [20]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [21]:
current_state = {
    "csv_file_path": csv_file_path,
    "output_dir": output_dir
}

In [22]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import create_mcp_state_tool_wrapper

In [23]:
bsm_state_tool = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir"
    },
    output_key="bsm_results_path"
)

In [24]:
result_update = bsm_state_tool(state=current_state)

In [25]:
result_update

{'bsm_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'}

## Agent Level

### calculator_tool_node

#### Node Inside Level

In [26]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [27]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents.mcp_client import create_mcp_state_tool_wrapper

In [28]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [29]:
errors = current_state.get("errors", [])
[
    "csv_file_path" not in current_state or not current_state["csv_file_path"],
    "server_path" not in current_state or not current_state["server_path"]
]

[False, False]

In [30]:
server_path = current_state["server_path"]
bsm_state_fn = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir",
    },
    output_key="bsm_results_path",
)

greeks_state_fn = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_greeks_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir",
    },
    output_key="greeks_results_path",
)

In [31]:
bsm_update = bsm_state_fn(state=current_state)
greeks_update = greeks_state_fn(state=current_state)

In [32]:
current_state["bsm_results_path"] = bsm_update["bsm_results_path"]
current_state["greeks_results_path"] = greeks_update["greeks_results_path"]

#### Node Level

In [33]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [34]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents import pricing_calculator_nodes
importlib.reload(pricing_calculator_nodes)
from bsm_multi_agents.agents.pricing_calculator_nodes import pricing_calculator_tool_node

In [35]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [38]:
update_state = pricing_calculator_tool_node(state=current_state)
update_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'bsm_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv',
 'greeks_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv',
 'errors': []}

### calculator_agent_node

#### Node Inside Level

In [39]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
prompt_path = os.path.join(project_path, "src/bsm_multi_agents/prompts/calculator_prompts.txt")

In [40]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.prompts.loader import load_prompt

In [41]:
current_state = WorkflowState(
        csv_file_path=csv_file_path, 
        output_dir=output_dir, 
        server_path=server_path
    )

In [42]:
errors = current_state.get("errors", [])
"csv_file_path" not in current_state or not current_state["csv_file_path"]

False

In [43]:
system_prompt = """
You are a quantitative calculator agent.
You have access to tools that call an external MCP server.
Each tool reads input file paths from the shared workflow state
and writes result file paths back into the state.
Do NOT ask for raw CSV content. Always work with paths only.
You do not need to provide any arguments to the tools. Just call them with {}.
"""
user_prompt_template = load_prompt(prompt_path)
formatted_user_prompt = (
    user_prompt_template.format(csv_file_path=current_state["csv_file_path"])
    if "{csv_file_path}" in user_prompt_template
    else user_prompt_template
)
llm = get_llm()
system_msg = SystemMessage(content=system_prompt)
human_msg = HumanMessage(content=formatted_user_prompt)
ai_msg = llm.invoke([system_msg, human_msg])

In [44]:
messages = list(current_state.get("messages", []))
messages.extend([system_msg, human_msg, ai_msg])
current_state["messages"] = messages

current_state["errors"] = errors

In [45]:
current_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\nYou are a quantitative calculator agent.\nYou have access to tools that call an external MCP server.\nEach tool reads input file paths from the shared workflow state\nand writes result file paths back into the state.\nDo NOT ask for raw CSV content. Always work with paths only.\nYou do not need to provide any arguments to the tools. Just call them with {}.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` tool to calculate prices.\n- Use the `calculate_greeks_to_file` tool to calc

#### Node Level

In [46]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [47]:
from bsm_multi_agents.agents import pricing_calculator_nodes
importlib.reload(pricing_calculator_nodes)
from bsm_multi_agents.agents.pricing_calculator_nodes import pricing_calculator_agent_node

from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

In [48]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [49]:
result_state = pricing_calculator_agent_node(current_state)

In [50]:
result_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\n    You are a quantitative calculator agent.\n    You have access to tools that call an external MCP server.\n    Each tool reads input file paths from the shared workflow state\n    and writes result file paths back into the state.\n    Do NOT ask for raw CSV content. Always work with paths only.\n    You do not need to provide any arguments to the tools. Just call them with {}.\n    ', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` tool to calculate prices.\n- Use the `calculate_

### Combined Tool and Agent Levle

In [51]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [52]:
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.pricing_calculator_nodes import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

In [53]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [54]:
current_state = pricing_calculator_agent_node(current_state)
current_state = pricing_calculator_tool_node(current_state)

In [55]:
current_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\n    You are a quantitative calculator agent.\n    You have access to tools that call an external MCP server.\n    Each tool reads input file paths from the shared workflow state\n    and writes result file paths back into the state.\n    Do NOT ask for raw CSV content. Always work with paths only.\n    You do not need to provide any arguments to the tools. Just call them with {}.\n    ', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` tool to calculate prices.\n- Use the `calculate_

# Validator

## MCP Tool Level